# Neural networks code and tests notebook

# This notebook takes the flux, photon index and fratio data and try to predict classes from this data on a :

# - LSTM network
# - ConvLSTM network
# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []

nbfeatures=3

a=binning
b=nbfeatures
c=lg

# multivariate

data_matrix= np.zeros((b,c,a))

#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][j]=dataframe['Flux']
    data_matrix[1][j]=dataframe['Photon Index']*dataframe['Flux']
    data_matrix[2][j]=dataframe['Photon Index']    
for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][v]=dataframe['Flux']
    data_matrix[1][v]=dataframe['Photon Index']*dataframe['Flux']
    data_matrix[2][v]=dataframe['Photon Index']     
#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))



In [6]:
#Reshaping data matrix
data_matrix=data_matrix.reshape(c,nbfeatures,binning)

In [7]:
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(300):
    
    model = Sequential()
    model.add(LSTM(9,input_shape=(nbfeatures,binning),activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])
    
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.1, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
980/980 [==============================] - 5s 2ms/step - loss: 0.5472 - binary_accuracy: 0.6750
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4532 - binary_accuracy: 0.6959
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4150 - binary_accuracy: 0.6959
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3943 - binary_accuracy: 0.6959
Iteration no. 0
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 5s 2ms/step - loss: 0.6242 - binary_accuracy: 0.5939
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4960 - binary_accuracy: 0.6908
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4369 - binary_accuracy: 0.6908
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4131 - binary_accuracy: 0.6908
Iteration no. 1
4/4 [==============================] - 1s 3ms/step
Epoch 1/5
980/980 

980/980 [==============================] - 3s 3ms/step - loss: 0.3941 - binary_accuracy: 0.6980
Iteration no. 33
4/4 [==============================] - 1s 3ms/step
Epoch 1/5
980/980 [==============================] - 5s 3ms/step - loss: 0.5420 - binary_accuracy: 0.6980
Epoch 2/5
980/980 [==============================] - 3s 3ms/step - loss: 0.4503 - binary_accuracy: 0.6959
Epoch 3/5
980/980 [==============================] - 3s 3ms/step - loss: 0.4130 - binary_accuracy: 0.6959
Iteration no. 34
4/4 [==============================] - 1s 3ms/step
Epoch 1/5
980/980 [==============================] - 5s 3ms/step - loss: 0.5654 - binary_accuracy: 0.6668
Epoch 2/5
980/980 [==============================] - 3s 3ms/step - loss: 0.4645 - binary_accuracy: 0.6888
Epoch 3/5
980/980 [==============================] - 3s 3ms/step - loss: 0.4268 - binary_accuracy: 0.6888
Epoch 4/5
980/980 [==============================] - 3s 3ms/step - loss: 0.3964 - binary_accuracy: 0.7041
Epoch 5/5
980/980 [=======

980/980 [==============================] - 2s 2ms/step - loss: 0.4170 - binary_accuracy: 0.6929
Iteration no. 67
4/4 [==============================] - 0s 3ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5577 - binary_accuracy: 0.8031
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4492 - binary_accuracy: 0.6867
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4115 - binary_accuracy: 0.6857
Iteration no. 68
4/4 [==============================] - 0s 3ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5608 - binary_accuracy: 0.7133
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4680 - binary_accuracy: 0.6949
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4258 - binary_accuracy: 0.6949
Iteration no. 69
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step

4/4 [==============================] - 1s 4ms/step
Epoch 1/5
980/980 [==============================] - 9s 5ms/step - loss: 0.5617 - binary_accuracy: 0.7026
Epoch 2/5
980/980 [==============================] - 5s 5ms/step - loss: 0.4588 - binary_accuracy: 0.6939
Epoch 3/5
980/980 [==============================] - 5s 5ms/step - loss: 0.4092 - binary_accuracy: 0.6949
Iteration no. 102
4/4 [==============================] - 1s 4ms/step
Epoch 1/5
980/980 [==============================] - 9s 5ms/step - loss: 0.6230 - binary_accuracy: 0.5735
Epoch 2/5
980/980 [==============================] - 5s 5ms/step - loss: 0.4897 - binary_accuracy: 0.6959
Epoch 3/5
980/980 [==============================] - 5s 5ms/step - loss: 0.4368 - binary_accuracy: 0.6959
Epoch 4/5
980/980 [==============================] - 5s 5ms/step - loss: 0.4103 - binary_accuracy: 0.6959
Iteration no. 103
4/4 [==============================] - 1s 4ms/step
Epoch 1/5
980/980 [==============================] - 9s 5ms/step - lo

980/980 [==============================] - 2s 2ms/step - loss: 0.4236 - binary_accuracy: 0.6969
Iteration no. 136
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5769 - binary_accuracy: 0.6122
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4416 - binary_accuracy: 0.7000
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4094 - binary_accuracy: 0.7000
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3914 - binary_accuracy: 0.7000
Iteration no. 137
4/4 [==============================] - 0s 1ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5798 - binary_accuracy: 0.6332
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4882 - binary_accuracy: 0.7000
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4313 - binary_accuracy: 0.6949
Epoch 4/5
980/980 [=====

980/980 [==============================] - 1s 2ms/step - loss: 0.4148 - binary_accuracy: 0.6949
Iteration no. 171
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5554 - binary_accuracy: 0.6974
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4580 - binary_accuracy: 0.6980
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4215 - binary_accuracy: 0.6980
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4145 - binary_accuracy: 0.6980
Iteration no. 172
4/4 [==============================] - 0s 1ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.6411 - binary_accuracy: 0.6439
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.5070 - binary_accuracy: 0.6878
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4489 - binary_accuracy: 0.6878
Epoch 4/5
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.4354 - binary_accuracy: 0.6867
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4238 - binary_accuracy: 0.6867
Iteration no. 207
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5521 - binary_accuracy: 0.6306
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4382 - binary_accuracy: 0.6929
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.3979 - binary_accuracy: 0.6929
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.3783 - binary_accuracy: 0.6929
Iteration no. 208
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5575 - binary_accuracy: 0.6658
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4594 - binary_accuracy: 0.6949
Epoch 3/5
980/980 [=====

980/980 [==============================] - 3s 1ms/step - loss: 0.5667 - binary_accuracy: 0.6526
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4648 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4246 - binary_accuracy: 0.6990
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4038 - binary_accuracy: 0.6990
Iteration no. 226
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.6184 - binary_accuracy: 0.8214
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4758 - binary_accuracy: 0.6939
Epoch 3/5
980/980 [==============================] - 1s 2ms/step - loss: 0.4075 - binary_accuracy: 0.6939
Iteration no. 227
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5777 - binary_accuracy: 0.6281
Epoch 2/5
980/980 [=====

4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5675 - binary_accuracy: 0.6566
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4560 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4215 - binary_accuracy: 0.6990
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.3994 - binary_accuracy: 0.6990
Iteration no. 262
4/4 [==============================] - 0s 1ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.5636 - binary_accuracy: 0.6842
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4693 - binary_accuracy: 0.6888
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4308 - binary_accuracy: 0.6888
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4154 - binary_accuracy: 0.6888
Iteration no. 263
4/4 [=========

Iteration no. 295
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 1ms/step - loss: 0.6184 - binary_accuracy: 0.7796
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4606 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4159 - binary_accuracy: 0.7510
Iteration no. 296
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5477 - binary_accuracy: 0.6944
Epoch 2/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4528 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4180 - binary_accuracy: 0.6990
Epoch 4/5
980/980 [==============================] - 1s 1ms/step - loss: 0.4004 - binary_accuracy: 0.6990
Iteration no. 297
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] 

In [8]:
# Conv/LSTM architecture
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(300):
    
    model = Sequential()
    model.add(Dense(binning, activation='tanh', input_shape=(nbfeatures,binning),input_dim=nbfeatures))
    model.add(Conv1D(filters=20, kernel_size=1, activation='tanh'))
    model.add(LSTM(20,input_shape=(20, nbfeatures), activation='tanh'))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='none'),optimizer='SGD', metrics=['binary_accuracy'])
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.1, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5566 - binary_accuracy: 0.6474
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4146 - binary_accuracy: 0.6959
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3851 - binary_accuracy: 0.6959
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3815 - binary_accuracy: 0.6959
Iteration no. 0
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5524 - binary_accuracy: 0.6684
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4273 - binary_accuracy: 0.6908
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4041 - binary_accuracy: 0.6908
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3889 - binary_accuracy: 0.6908
Iteration no. 1
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 

980/980 [==============================] - 3s 2ms/step - loss: 0.5353 - binary_accuracy: 0.7194
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4182 - binary_accuracy: 0.6939
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3897 - binary_accuracy: 0.6939
Iteration no. 36
4/4 [==============================] - 0s 0s/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5051 - binary_accuracy: 0.7097
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4033 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3842 - binary_accuracy: 0.6990
Iteration no. 37
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5574 - binary_accuracy: 0.7015
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4200 - binary_accuracy: 0.6918
Epoch 3/5
980/980 [========

4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5113 - binary_accuracy: 0.7015
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4054 - binary_accuracy: 0.6918
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3775 - binary_accuracy: 0.6918
Iteration no. 71
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5143 - binary_accuracy: 0.7245
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4075 - binary_accuracy: 0.7000
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3813 - binary_accuracy: 0.7000
Iteration no. 72
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5426 - binary_accuracy: 0.6245
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss

980/980 [==============================] - 3s 2ms/step - loss: 0.5049 - binary_accuracy: 0.6980
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4030 - binary_accuracy: 0.6980
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3839 - binary_accuracy: 0.6980
Iteration no. 107
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5660 - binary_accuracy: 0.6367
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4163 - binary_accuracy: 0.6990
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3862 - binary_accuracy: 0.6990
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3761 - binary_accuracy: 0.6990
Iteration no. 108
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 3s 2ms/step - loss: 0.5576 - binary_accuracy: 0.7117
Epoch 2/5
980/980 [=====

980/980 [==============================] - 2s 2ms/step - loss: 0.3866 - binary_accuracy: 0.6949
Iteration no. 142
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5169 - binary_accuracy: 0.7128
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4050 - binary_accuracy: 0.7082
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3879 - binary_accuracy: 0.7000
Iteration no. 143
4/4 [==============================] - 1s 93ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5378 - binary_accuracy: 0.6699
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4024 - binary_accuracy: 0.6959
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3784 - binary_accuracy: 0.6959
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3705 - binary_accuracy: 0.6959
Iteration no. 144
4/4 [

980/980 [==============================] - 2s 2ms/step - loss: 0.3889 - binary_accuracy: 0.6980
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3813 - binary_accuracy: 0.6980
Iteration no. 178
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5423 - binary_accuracy: 0.6122
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4142 - binary_accuracy: 0.6918
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3941 - binary_accuracy: 0.6918
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3869 - binary_accuracy: 0.6918
Iteration no. 179
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5111 - binary_accuracy: 0.6342
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3969 - binary_accuracy: 0.6929
Epoch 3/5
980/980 [=====

980/980 [==============================] - 4s 2ms/step - loss: 0.5155 - binary_accuracy: 0.6821
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4023 - binary_accuracy: 0.6949
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3814 - binary_accuracy: 0.6949
Epoch 4/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3730 - binary_accuracy: 0.6949
Iteration no. 214
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5374 - binary_accuracy: 0.6990
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4108 - binary_accuracy: 0.6939
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3874 - binary_accuracy: 0.6939
Iteration no. 215
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5318 - binary_accuracy: 0.7056
Epoch 2/5
980/980 [=====

4/4 [==============================] - 0s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5610 - binary_accuracy: 0.7332
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4299 - binary_accuracy: 0.6908
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4028 - binary_accuracy: 0.6908
Iteration no. 250
4/4 [==============================] - 1s 2ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5274 - binary_accuracy: 0.7617
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - loss: 0.4072 - binary_accuracy: 0.6898
Epoch 3/5
980/980 [==============================] - 2s 2ms/step - loss: 0.3853 - binary_accuracy: 0.6898
Iteration no. 251
4/4 [==============================] - 0s 3ms/step
Epoch 1/5
980/980 [==============================] - 4s 2ms/step - loss: 0.5295 - binary_accuracy: 0.6663
Epoch 2/5
980/980 [==============================] - 2s 2ms/step - lo

980/980 [==============================] - 3s 3ms/step - loss: 0.4129 - binary_accuracy: 0.7010
Epoch 3/5
980/980 [==============================] - 3s 3ms/step - loss: 0.3916 - binary_accuracy: 0.7010
Iteration no. 284
4/4 [==============================] - 1s 3ms/step
Epoch 1/5
980/980 [==============================] - 7s 4ms/step - loss: 0.5189 - binary_accuracy: 0.7031
Epoch 2/5
980/980 [==============================] - 3s 3ms/step - loss: 0.4097 - binary_accuracy: 0.6898
Epoch 3/5
980/980 [==============================] - 3s 3ms/step - loss: 0.3917 - binary_accuracy: 0.6898
Iteration no. 285
4/4 [==============================] - 1s 3ms/step
Epoch 1/5
980/980 [==============================] - 6s 3ms/step - loss: 0.5236 - binary_accuracy: 0.6638
Epoch 2/5
980/980 [==============================] - 3s 3ms/step - loss: 0.3936 - binary_accuracy: 0.7031
Epoch 3/5
980/980 [==============================] - 3s 3ms/step - loss: 0.3790 - binary_accuracy: 0.7031
Epoch 4/5
980/980 [=====

In [9]:
from sklearn.metrics import f1_score

ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(300):
    nb_filters=20
    
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)

    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="tanh",input_shape=(nbfeatures,binning)))
    model.add(Conv1D(15, 1, padding="same", activation="tanh",input_shape=(nb_filters,binning)))
    model.add(Conv1D(9, 1, padding="same", activation="tanh",input_shape=(15,binning)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(9, activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.5162 - binary_accuracy: 0.7388
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4043 - binary_accuracy: 0.7130
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.3873 - binary_accuracy: 0.7130
Iteration no. 0
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.4858 - binary_accuracy: 0.6831
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4144 - binary_accuracy: 0.6912
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4027 - binary_accuracy: 0.7348
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.3985 - binary_accuracy: 0.6912
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.3941 - binary_accuracy: 0.6912
Iteration no. 1
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 

7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4790 - binary_accuracy: 0.6722
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.4126 - binary_accuracy: 0.7015
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.3864 - binary_accuracy: 0.6935
Epoch 4/5
871/871 [==============================] - 1s 2ms/step - loss: 0.3771 - binary_accuracy: 0.6935
Iteration no. 17
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5290 - binary_accuracy: 0.6986
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.3977 - binary_accuracy: 0.6923
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.3953 - binary_accuracy: 0.6923
Iteration no. 18
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 2s 2ms/step - loss

871/871 [==============================] - 2s 2ms/step - loss: 0.4176 - binary_accuracy: 0.6946
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4036 - binary_accuracy: 0.6935
Iteration no. 52
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4959 - binary_accuracy: 0.7560
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4008 - binary_accuracy: 0.7176
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.3861 - binary_accuracy: 0.6889
Iteration no. 53
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4906 - binary_accuracy: 0.7394
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.4136 - binary_accuracy: 0.7290
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4128 - binary_accuracy: 0.6854
Iteration no. 54
7/7 [====

871/871 [==============================] - 3s 3ms/step - loss: 0.3669 - binary_accuracy: 0.6912
Iteration no. 85
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 5s 3ms/step - loss: 0.4866 - binary_accuracy: 0.6705
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4256 - binary_accuracy: 0.6912
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4045 - binary_accuracy: 0.6900
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.3964 - binary_accuracy: 0.6900
Iteration no. 86
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4497 - binary_accuracy: 0.6866
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.4015 - binary_accuracy: 0.6935
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3813 - binary_accuracy: 0.6935
Epoch 4/5
871/871 [=======

7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4959 - binary_accuracy: 0.6877
Epoch 2/5
871/871 [==============================] - 2s 3ms/step - loss: 0.3837 - binary_accuracy: 0.7015
Epoch 3/5
871/871 [==============================] - 2s 3ms/step - loss: 0.3750 - binary_accuracy: 0.6992
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3745 - binary_accuracy: 0.6992
Iteration no. 121
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.5089 - binary_accuracy: 0.6361
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3905 - binary_accuracy: 0.7348
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3941 - binary_accuracy: 0.6831
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3862 - binary_accuracy: 0.6831
Iteration no. 122
7/7 [=========

871/871 [==============================] - 3s 3ms/step - loss: 0.3864 - binary_accuracy: 0.7245
Epoch 4/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3914 - binary_accuracy: 0.6980
Epoch 5/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3866 - binary_accuracy: 0.6980
Iteration no. 139
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.4713 - binary_accuracy: 0.7164
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.4097 - binary_accuracy: 0.7118
Epoch 3/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3968 - binary_accuracy: 0.6900
Iteration no. 140
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 4s 3ms/step - loss: 0.4713 - binary_accuracy: 0.7704
Epoch 2/5
871/871 [==============================] - 3s 3ms/step - loss: 0.3798 - binary_accuracy: 0.7038
Epoch 3/5
871/871 [=====

871/871 [==============================] - 4s 5ms/step - loss: 0.3777 - binary_accuracy: 0.6980
Iteration no. 174
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 6s 5ms/step - loss: 0.4850 - binary_accuracy: 0.7842
Epoch 2/5
871/871 [==============================] - 4s 5ms/step - loss: 0.4185 - binary_accuracy: 0.7038
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4023 - binary_accuracy: 0.6923
Iteration no. 175
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 5s 4ms/step - loss: 0.4947 - binary_accuracy: 0.6860
Epoch 2/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4228 - binary_accuracy: 0.6889
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4130 - binary_accuracy: 0.6889
Epoch 4/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4065 - binary_accuracy: 0.6889
Iteration no. 176
7/7 [=

871/871 [==============================] - 6s 5ms/step - loss: 0.4956 - binary_accuracy: 0.6757
Epoch 2/5
871/871 [==============================] - 4s 5ms/step - loss: 0.4190 - binary_accuracy: 0.6831
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4057 - binary_accuracy: 0.6820
Epoch 4/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4006 - binary_accuracy: 0.6831
Iteration no. 209
7/7 [==============================] - 0s 4ms/step
Epoch 1/5
871/871 [==============================] - 6s 5ms/step - loss: 0.5011 - binary_accuracy: 0.7606
Epoch 2/5
871/871 [==============================] - 4s 5ms/step - loss: 0.3936 - binary_accuracy: 0.6992
Epoch 3/5
871/871 [==============================] - 4s 5ms/step - loss: 0.3792 - binary_accuracy: 0.6992
Iteration no. 210
7/7 [==============================] - 0s 4ms/step
Epoch 1/5
871/871 [==============================] - 6s 5ms/step - loss: 0.4567 - binary_accuracy: 0.6854
Epoch 2/5
871/871 [=====

871/871 [==============================] - 4s 4ms/step - loss: 0.3938 - binary_accuracy: 0.7003
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.3850 - binary_accuracy: 0.7003
Iteration no. 244
7/7 [==============================] - 0s 3ms/step
Epoch 1/5
871/871 [==============================] - 5s 4ms/step - loss: 0.4795 - binary_accuracy: 0.6843
Epoch 2/5
871/871 [==============================] - 4s 4ms/step - loss: 0.4045 - binary_accuracy: 0.6935
Epoch 3/5
871/871 [==============================] - 4s 4ms/step - loss: 0.3969 - binary_accuracy: 0.6935
Epoch 4/5
871/871 [==============================] - 4s 4ms/step - loss: 0.3915 - binary_accuracy: 0.6935
Iteration no. 245
7/7 [==============================] - 0s 4ms/step
Epoch 1/5
871/871 [==============================] - 5s 4ms/step - loss: 0.4932 - binary_accuracy: 0.7239
Epoch 2/5
871/871 [==============================] - 3s 4ms/step - loss: 0.4162 - binary_accuracy: 0.6969
Epoch 3/5
871/871 [=====

7/7 [==============================] - 0s 4ms/step
Epoch 1/5
871/871 [==============================] - 6s 5ms/step - loss: 0.5097 - binary_accuracy: 0.6602
Epoch 2/5
871/871 [==============================] - 4s 5ms/step - loss: 0.4150 - binary_accuracy: 0.6980
Epoch 3/5
871/871 [==============================] - 5s 6ms/step - loss: 0.4074 - binary_accuracy: 0.6980
Epoch 4/5
871/871 [==============================] - 6s 7ms/step - loss: 0.3878 - binary_accuracy: 0.6980
Iteration no. 280
7/7 [==============================] - 0s 4ms/step
Epoch 1/5
871/871 [==============================] - 8s 6ms/step - loss: 0.4670 - binary_accuracy: 0.6923
Epoch 2/5
871/871 [==============================] - 5s 5ms/step - loss: 0.3997 - binary_accuracy: 0.6935
Epoch 3/5
871/871 [==============================] - 5s 6ms/step - loss: 0.3843 - binary_accuracy: 0.6935
Epoch 4/5
871/871 [==============================] - 5s 6ms/step - loss: 0.3825 - binary_accuracy: 0.6935
Iteration no. 281
7/7 [=========

In [ ]:
from timeseries.all import *

#Index of time series to choose for the visualization of CAM
idxs = [0,3]
#Get time series under a tensor form - get data batch
x, y = get_batch(dls.train.dataset, idxs)

figs=plt.figure()
for i in range(nbfeatures):
    
show_cam(batch, model, layer=5, i2o=i2o, func_cam=grad_cam_acts)
